In [58]:
from segment_anything.modeling.common import LayerNorm2d

import pickle
import torch
import sys

# Setup
### Hashing

In [59]:
import hashlib

def hash_tensor(tensor: torch.Tensor):
    data = str(tensor.flatten().tolist())
    return str(hashlib.md5(data.encode()).hexdigest())

### Data formatting

In [60]:
import json
import os

class Item:
    def __init__(self, key, value, type):
        self.key = key
        self.type = type
        if type == "Tensor":
            self.value = {"size":value.size(),"hash":hash_tensor(value)}
        else:
            self.value = value

    def to_dict(self):
        return {self.key: {self.type: self.value}}


def to_file(name:str,items:list):
    path = "test-files/"+name+".json"
    values = {}
    for item in items:
        values.update(item.to_dict())
    output = {"values": values}
    
    data = json.dumps(output, indent=4)
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w") as f:
        f.write(data)
        
def random_tensor():
    return torch.ones(2,256,16,16)


## Common
#### LayerNorm2d

In [72]:
# New
layer_norm = LayerNorm2d(256,0.1)
items = [Item("weight", layer_norm.weight, "Tensor"), Item("bias", layer_norm.bias, "Tensor"), Item("eps", layer_norm.eps, "Float")]
to_file("layer_norm_2d",items)

# Forward
input = random_tensor()
output = layer_norm(input)
items = [Item("input", input, "Tensor"), Item("output", output, "Tensor")]
to_file("layer_norm_2d_forward",items)